# SOLVERS ONLINE

## PREPARING THE DEVELOPMENT ENVIRONMENT

Importing the libraries:

In [1]:
from numpy.random import shuffle
from collections import defaultdict

Auxiliary function to print the maze and the intelligent agent:

- `(x, y)` represents the position of the agent

In [2]:
def is_conn(graph, v, w):
    return v in graph[w]

def get_character(maze, x, y, r, c):
    return 'X' if x == c and y == r else 'O'

def get_row(maze, r, c, n):
    v = n * r + c

    return ' ' if is_conn(maze, v, v+1) else '|'

def get_col(maze, r, c, n):
    v = n * r + c

    return '  ' if is_conn(maze, v, v+n) else '_ '

def print_maze(n, maze, x, y):
    for r in range(n):
        for c in range(n-1):
            print(get_character(maze, x, y, r, c),
                  end=get_row(maze, r, c, n))

        print(get_character(maze, x, y, r, c+1))

        if r != n-1:
            for c in range(n):
                print(end=get_col(maze, r, c, n))

            print()

## MAZE GENERATOR

Replicating the maze generator:

> **Note**: For detailed implementation see `notebooks/maze_generator.ipynb`.

In [3]:
class UnionFind:
    def __init__(self, n):
        self.n = n
        self.v = list(range(n))

    def find(self, u):
        while u != self.v[u]:
            self.v[u] = self.v[self.v[u]]
            u = self.v[u]

        return u

    def union(self, u, v):
        root_u, root_v = self.find(u), self.find(v)

        if root_u == root_v:
            return False
        else:
            self.v[root_v] = root_u
            self.n -= 1

            return True

class MazeGenerator:
    def __init__(self, n=100):
        self.n = n
        self.shape = int(n**0.5)

        self.edges = self.generate_edges()

    def moves(self, v):
        moves = []

        if (v + 1) % self.shape:
            moves.append((v, v + 1))
        if v + self.shape < self.n:
            moves.append((v, v + self.shape))

        return moves

    def generate_edges(self):
        return [edge
                for v in range(self.n)
                for edge in self.moves(v)]

    def generate_maze(self):
        forest = UnionFind(self.n)
        maze   = [[] for _ in range(self.n)]

        shuffle(self.edges)

        for v, w in self.edges:
            if forest.union(v, w):
                maze[v].append(w)
                maze[w].append(v)

                if forest.n == 1:
                    return maze

## MAZE LOGIC

Implementing maze logic:

- Class responsible for the maze
- The constructor generates a new grid that represents a maze
- Default is a 10x10 grid
- Getitem checks for the existence of an edge between two vertices

In [4]:
class Maze:
    def __init__(self, n=100):
        self.n = n
        self.shape = int(n**0.5)

        self.grid  = MazeGenerator(self.n).generate_maze()

    def __getitem__(self, move):
        pre, pos = move

        if 0 <= pos < self.n:
            return pre in self.grid[pos]

        return False

    def won(self, x, y):
        return x == y == self.shape-1

Developing the class responsible for the game character:

- Starts at position `(0, 0)`

In [5]:
class Fiancee:
    def __init__(self):
        self.x = 0
        self.y = 0

    def move(self, move):
        if (move == 'u'):
            self.y -= 1
        elif (move == 'd'):
            self.y += 1
        elif (move == 'l'):
            self.x -= 1
        elif (move == 'r'):
            self.x += 1

    @property
    def xy(self):
        return self.x, self.y

## ONLINE DFS AGENT

Defining constants and a dictionary that pairs movements with their corresponding reversals:




In [6]:
UNSUCCESS = -1
PERFORM = 0
SUCCESS = 1

undone = {'u' : 'd',
          'd' : 'u',
          'r' : 'l',
          'l' : 'r'}

Implementing the online dfs agent:

- Step online function: The `step_online` function takes a percept s as an argument, which identifies the current state of the agent within the environment
- Ensuring search status: Within the `step_online` function, it's crucial to verify the search status to ensure that the agent's exploration or search process is correctly ongoing

In [7]:
class OnlineDFSAgent:
    def __init__(self):
        self.result  = defaultdict(lambda: None)
        self.untried = defaultdict(list)
        self.unback  = defaultdict(list)

        self.s = None
        self.a = None

        self.status = PERFORM

    def step_online(self, maze, s):
        if self.status != PERFORM:
            return None
        if maze.won(*s):
            self.status = SUCCESS
            return None

        if not self.step(maze, s):
            self.status = UNSUCCESS

        return self.a

    def step(self, maze, s):
        if s not in self.untried:
            self.untried[s] = self.actions(maze, s)

        if self.s and s != self.result[self.s, self.a]:
            self.result[self.s, self.a]    = s
            self.result[s, undone[self.a]] = self.s

            self.unback[s].append(undone[self.a])

        self.s = s
        if self.untried[s]:
            self.a = self.untried[s].pop()
        elif self.unback[s]:
            self.a = self.unback[s].pop()
        else:
            self.a = None

        return self.a

    def actions(self, maze, p):
        actions = []

        pos = p[0] + maze.shape*p[1]

        if self.a != 'r' and maze[pos, pos-1]:
            actions.append('l')
        if self.a != 'l' and maze[pos, pos+1]:
            actions.append('r')
        if self.a != 'd' and maze[pos, pos-maze.shape]:
            actions.append('u')
        if self.a != 'u' and maze[pos, pos+maze.shape]:
            actions.append('d')

        return actions

    @property
    def path(self):
        path = [self.s]

        p = path[-1]
        while p != (0, 0):
            p = self.result[p, self.unback[p][0]]
            path.append(p)

        return path[::-1]

Testing the agent:

In [8]:
maze  = Maze()
agent = OnlineDFSAgent()
fianc = Fiancee()

print_maze(maze.shape, maze.grid, *fianc.xy)

steps = 0
while agent.status == PERFORM:
    act = agent.step_online(maze, fianc.xy)

    if act:
        fianc.move(act)
        steps += 1

print()
print()

print_maze(maze.shape, maze.grid, *fianc.xy)

X O O|O O|O O O O O
      _   _ _ _   _ 
O|O|O|O|O O O O O|O
_ _     _ _   _     
O O|O|O O|O|O|O O O
_     _     _   _ _ 
O O|O O O O O O O O
_       _ _ _ _ _   
O O|O|O O O|O O|O O
_   _ _   _   _   _ 
O O O O O|O|O O|O|O
  _           _ _   
O O|O|O|O|O|O O O O
_   _ _ _     _ _   
O O O O O O O O O|O
  _ _               
O|O O O|O|O|O|O|O|O
          _         
O|O|O|O|O O|O|O|O|O


O O O|O O|O O O O O
      _   _ _ _   _ 
O|O|O|O|O O O O O|O
_ _     _ _   _     
O O|O|O O|O|O|O O O
_     _     _   _ _ 
O O|O O O O O O O O
_       _ _ _ _ _   
O O|O|O O O|O O|O O
_   _ _   _   _   _ 
O O O O O|O|O O|O|O
  _           _ _   
O O|O|O|O|O|O O O O
_   _ _ _     _ _   
O O O O O O O O O|O
  _ _               
O|O O O|O|O|O|O|O|O
          _         
O|O|O|O|O O|O|O|O|X


In [9]:
print('Size of the path =', len(agent.path) + 1)
print('Number of steps  =', steps)
print('Final position   =', fianc.xy)

Size of the path = 29
Number of steps  = 88
Final position   = (9, 9)


## LRTA* agent

Implementing the LRTA* agent:

In [37]:
class LRTAStar:
    def __init__(self):
        self.results = defaultdict(lambda: None)
        self.H = dict()
        
        self.s = None
        self.a = None
        
        self.status = NOT_STARTED
        
    # The s_line argument is a percept that identifies the current state
    def step_online(self, maze, s_line):
        # Checking if it found a goal state
        if maze.won(*s_line):
            self.status = SUCCESS
        
        # Identifying search status
        if self.status == SUCCESS:
            print("Goal already found!")
            return None
        elif self.status == UNSUCCESS:
            print("Unsuccessful search!")
            return None
        elif self.status == NOT_STARTED:
            self.status = STARTED
        
        if s_line not in self.H:
            self.H[s_line] = self.manhattan(maze, *s_line)
            
        if self.s:
            self.results[self.s, self.a] = s_line
            
            self.H[self.s] = min([self.LRTA_cost(maze, self.s, self.results[self.s, b])
                                  for b in self.actions(maze, self.s)])
            
        self.s = s_line
        self.a = min([b for b in self.actions(maze, s_line)],
                     key=lambda b: self.LRTA_cost(maze, s_line, self.results[s_line, b]))        
        
        return self.a
            
    def actions(self, maze, pos):
        place = pos[0] + maze.shape*pos[1]
        
        actions = []
        
        if maze[place, place - maze.shape]:
            actions.append('u')
        if maze[place, place + 1]:
            actions.append('r')
        if maze[place, place + maze.shape]:
            actions.append('d')
        if maze[place, place - 1]:
            actions.append('l')

        return actions
    
    def LRTA_cost(self, maze, s, s_line):
        return 1 + self.H[s_line] if s_line else self.manhattan(maze, *s)
    
    @staticmethod
    def manhattan(maze, x, y):
        return abs(x - (maze.shape-1)) + abs(y - (maze.shape-1))  # manhattan heuristic

Testing the agent:

In [38]:
agent = LRTAStar()
fiancee = Fiancee()

steps = 0
while agent.status != SUCCESS and agent.status != UNSUCCESS:
    # Movement of agents
    action = agent.step_online(maze, fiancee.xy)
    fiancee.move(action)
    
    # Printing the route
    print_maze(maze.grid, fiancee.x, fiancee.y)
    print("\n")
    
    steps += 1

NameError: name 'NOT_STARTED' is not defined

x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x o x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x o|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - 

x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x o|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x o x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x

x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x o|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x o x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x o x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - - 

      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|o
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|o
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x

-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x o x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x o x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|o|x x x
      - - - -

      - - - -       
o|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
o|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
o|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -

print(f'Final position = {fiancee.xy}')
print(f'Number of steps = {steps}')

In [ ]:
%%timeit

agent = OnlineDFSAgent()
fiancee = Fiancee()

#print_maze(maze.shape, maze.grid, *fiancee.xy)

steps = 0
while True:
    action = agent.step_online(maze, fiancee.xy)
    if action:
        fiancee.move(action)
    else:
        break
    
    steps += 1

#print()
#print_maze(maze.shape, maze.grid, *fiancee.xy)